In [168]:
import pandas as pb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import os.path
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
import gzip

In [157]:
X = []
Y = []

for root, dirs, files in os.walk('Recording Data'):
    for file in files:
        if file.endswith(".txt") and file != 'Subject codes.txt':
            file_path = os.path.join(root, file)
            match root[-1]:
                case '1' | '2':
                    has_dyslexia = 1
                case _:
                    has_dyslexia = 0
            data = pd.read_csv(file_path, sep='\t', index_col=None, decimal=",")
            data['r_dist'] = (data['RX'].diff() ** 2 + data['RY'].diff() ** 2) ** 0.5
            data['l_dist'] = (data['LX'].diff() ** 2 + data['LY'].diff() ** 2) ** 0.5
            added_data = list(zip(data['r_dist'][1:], data['l_dist'][1:]))
            X.append(added_data)
            Y.append(has_dyslexia)

In [158]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [159]:
k_m = [1, 5, 10, 15, 20, 25]

In [4]:
#Перевести все в массивы numpy и вместо сложения использовать np.concatenate

In [160]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

f_scores = []
for k in k_m:
    for train_index, test_index in kf.split(X):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        Y_train, Y_test = [Y[i] for i in train_index], [Y[i] for i in test_index]

        predicts = []
        for x1 in X_test:
            x1 = np.array(x1)
            Cx1 = len(gzip.compress(x1))
            distance_from_x1 = []
            for x2 in X_train:
                x2 = np.array(x2)
                Cx2 = len(gzip.compress(x2))
                x1x2 = np.concatenate((x1, x2))
                Cx1x2 = len(gzip.compress(np.array(x1x2)))
                ncd = (Cx1x2 - min(Cx1 , Cx2)) / max (Cx1 , Cx2 )
                distance_from_x1.append(ncd)
                        
            sorted_idx = np.argsort(np.array(distance_from_x1))
            top_k_class = np.array(Y_train)[sorted_idx[:k]].tolist()
            predict_class = max(set(top_k_class), key = top_k_class.count)
            predicts.append(predict_class)
        
        f1 = classification_report(Y_test, predicts, output_dict=True)['macro avg']['f1-score']
        f_scores.append(f1)

In [161]:
f_scores_split = np.array_split(f_scores, 6)

In [165]:
table = pd.DataFrame(f_scores_split, index=[1, 5, 10, 15, 20, 25], columns=['1_split', '2_split', '3_split', '4_split', '5_split'])

In [166]:
table['avg_score'] = table.sum(axis=1) / 5

In [167]:
table

,1_split,2_split,3_split,4_split,5_split,avg_score
1,0.863267,0.854216,0.756757,0.779762,0.864469,0.823694
5,0.864469,0.912530,0.891813,0.891176,0.891176,0.890233
10,0.889881,0.805701,0.808574,0.864865,0.918681,0.857540
15,0.945588,0.858238,0.837719,0.918681,0.945906,0.901227
20,0.917960,0.805701,0.836765,0.891813,0.863267,0.863101
25,0.889881,0.912530,0.834821,0.864865,0.891176,0.878655
